<a href="https://colab.research.google.com/github/harshini2001/Dots_And_Boxes_RL/blob/main/version3_0_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bitstring

     |████████████████████████████████| 204kB 18.2MB/s 
  Created wheel for bitstring: filename=bitstring-3.1.7-cp37-none-any.whl size=37949 sha256=b90b72ed261be672299c8538d917eb65ccdddf66a5e4ffc095776a545f23c80d
  Stored in directory: /root/.cache/pip/wheels/b8/27/f0/8373e26b7de57db03dc18aaaebdd8c26a99da882416f762979
Successfully built bitstring


## ML ASSIGNMENT BATCH 8
- Deepthishree GS
- Harshini S
- Iswarya GP
- Janani R
- Swetha M

### ***TOPIC*** - REINFORCEMENT LEARNING
### ***Problem Taken*** : Playing Dots and Boxes using a Q learning Agent

![image](https://user-images.githubusercontent.com/43994542/112790879-f9220480-907d-11eb-8dae-c041684a9523.png)



## Importing Libraries 

In [2]:
from bitstring import BitArray
# This library is used to store and convert the board state. 
# Board state is stored as bitarray with one indicating a line and zero indicating no line
import numpy as np
import random
# To make random moves in initial exploration phase
import json
# Save the QTABLE finally as a json file to continue training incrementally when the kernal restarts

## Global Variables declaration


In [3]:
'''
3x3 dots and boxes 
4 boxes can be drawn
12 lines to choose from for each move
2 power 12 game states for the agent to learn from
DRAW / TIE is available in this version of the game (in original version in case of tie, person who starts loses)
Reward 100 and Penalty -100 (for each box drawn)
Reward 200 and Penalty -200 (long term finally at the end)
Reward 50 for a tie game at end
'''
QTable=dict()
# This table is the ultimate QTable to be learnt by our agent
'''
Q TABLE usually has states as rows and actions as columns and the entries are the corresponding Q values

Since in a game, only legal moves are the actions, the number of columns(moves) for each state is different

So, a hash table is constructed with keys as states and values as action:qvalue pairs
'''
VISIT_TABLE=dict()
# This table has the count of visits for each state. This ensures that we can check if all states are visited often (An assumption for Q Learning Convergence in TOM MITCHELL book)
DISCOUNT_RATE=0.8
# This decreases the future rewards by a factor each step
LEARNING_RATE=0.2
# This makes sure table is learnt at a specified. A balance between existing value and updated value
# Small LRate values ensure the table is not drastically updated


### Main Function for LEARNER
- Algorithm is play games repeatedly and update the Q Table 
- Calls subfunction playGame

#### Main Functions:
- Decide the choice of move for agent as random, simple or qlearner 
- Ensures where the learner is exploring or exploiting
- Random and simple for exploration
  - Random makes random moves
  - Simple makes the move with least visit count for the next state (Thus makes sure the same state is not repeated often and unvisited states are explored)
- Qlearner exploits
  - This makes the move where the Q value for the next state is maximum
  

In [19]:
def learner(gameCount=1000): #default no of games if the trainer doesn't specify the games

    count=1
    # counter to keep track of number of games
    choiceOfMove=['random','simple','qlearner']
    # choice for deciding exploration or exploitation
    while count<gameCount:
        print("GAME ",count)
        #print the game number to know the progress of learner
        if count<gameCount/10:
            playGame(choiceOfMove[0],0,False)
        #First 10% games are random
        elif count<gameCount/2:
            playGame(choiceOfMove[1],0,False)
        #Till 50% the games are moved by a simple agent - Exploring the unvisited states
        else:
            playGame(choiceOfMove[2],0,False)
        #Last 50% of games is exploited by QLearner
        count+=1
        with open('Qtable.json','w') as jsonfile:
          json.dump(QTable,jsonfile)
        # Each game, the current QTABLE is backed up incase the kernel restarts and values are lost



    
        
        

### Initialises the Main Q Table
- Q TABLE usually has states as rows and actions as columns and the entries are the corresponding Q values

- Since in a game, only legal moves are the actions, the number of columns(moves) for each state is different

- So, a hash table is constructed with keys as states and values as action:qvalue pairs

- All values are init to 0 now


In [20]:

def initialiseQTable():
    global QTable
    global VISIT_TABLE
    # The global variables are initialised in this function
    for i in range(4096):
      '''
      The number of states is 2 power 12 which is 4096
      QTABLE rows (in our case the keys of the hashtable are 4096 in number )

      For each state, each line which can be drawn is a possible action. 
      Initial state = 000000000000 => has 12 possible actions
      Some state    = 000011110000 => has 8 zeros and 4 ones. 
                      Out of the 12 lines, 4 are drawn. 8 lines are free to be drawn for next move.
                      The possible line numbers which can be drawn are the possible actions
                      EACH ZERO is a possible action. Here, 1,2,3,4 and 9,10,11,12 are the 8 possible actions. 
      So, the actions are initialised as another dictionary and 0 is the current initialisation for Q value
      '''
      currentState=BitArray(uint=i, length=12)
      actionQvalues=dict()
      # find possible actions
      bitString=currentState.bin
      # convert bitarray to bitstring to store as key
      for index in range(len(bitString)):
          if bitString[index]=='0':
              actionQvalues[index+1]=0
            
          #EACH ZERO in this state is a possible action when making next move from this state
          # index + 1 because line number index starts from 1 for ease of human player

      QTable[currentState.bin]=actionQvalues
      VISIT_TABLE[currentState.bin]=0
      # EACH State visit count is init to 0 
      QTable['111111111111']={0:0}
      # FINAL BOARD STATE HAS ONLY ONE POSSIBLE ACTION - NO MOVE - 0 is set as action which is not a valid line number (line numbers start from 1 to 12)


                
        
    

### Best action given a state
Function used by QLearner to select the action or move which has maximum QValue for all possible actioins in the given state

In [21]:
#This is the function to choice best action for the  given state using Qvalue in QTable
def bestActionForState(state):
    maxQval=-2000 #negative inf
    bestAction=0
    for action,qval in QTable[state].items():
        # QTABLE has all valid actions for current state and their QValues
        if qval>maxQval:
            maxQval=qval # Find the maximum qvalue
            bestAction=action # Find the corresponding best action
    return bestAction



    
    

### UPDATE QTABLE
- This function uses the formula for QLearning and updates the moves in reverse chronological order
- This is called after each game is over
- This finetuning is what makes the agent learn the game
- According to Tom Mitchell book, update for QTable values is as 
  `Q(s,a) = (1-LR)*Q(s,a)+LR*(rwd+DR*MAX(Q(ns,all a's)))`
  - First term is existing term
  - Second term is update term
  - LR and DR are learning and discount rate respectively

In [22]:

def updateQTable(memoryOfPlayerMoves):
    global QTable
    memoryOfPlayerMoves=memoryOfPlayerMoves[::-1] #reverse chronological order
    '''
    The first few moves have no rewards. According to Tom Mitchell book, the sequence of updates have no impact. 
    Therefore, it is efficient to propagate the effect of rewards from current move to the previous moves. 
    This helps us indirectly since we have rewards only for end result and certain important points in game.
    For remaining moves the immediate reward is 0
    '''
    for state,action,nextState,reward in memoryOfPlayerMoves:
        maxQValForNextState=QTable[nextState][bestActionForState(nextState)]
        # Finding the maximum QValue for next state to update future rewards along with a discount
        existingValue=(1-LEARNING_RATE)*QTable[state][action]
        # Existing value is taken by a fraction as indicated by learning rate to prevent drastic updates in Qtable
        updateValue=LEARNING_RATE*(reward+DISCOUNT_RATE*maxQValForNextState)
        # UPDATE value is immediate reward + Discount rate * future rewards
        # ACCORDING TO an equation in Tom Mitchell book
        QTable[state][action]=existingValue+updateValue
        VISIT_TABLE[state]+=1
        #Since this state is visited, its visit count is incremented
        

### Print board state
- Print the board state so that the human can understand what are the boxes formed and what could be the potential next move during a game

In [23]:
#Printing 3*3 table
def printTable(currentState):
    count=1
    m=7
    for i in range(3):
        for j in range(3):
            print(".\t", end=" ")
            if j<2:
                if currentState[count-1]=='0':
                    print(str(count)+"\t",end=" ") 
                else:
                    print("--\t",end=" ")
                count+=1
                
        print("\n")
        if i!=2:
            for k in range(3):
                if currentState[m-1]=='0':
                    print(str(m)+"\t\t",end=" ")
                else:
                    print("|\t\t",end=" ")
                m+=1
        print("\n")
# printTable('111111111111')

### MakeMove or ChooseAction
- First few games, learner uses random moves
- Next it uses an approach to select states with very less visit count. 
- These are for exploration
- After that, the game is carried forward by the QLearner

- Human can choose any of the three for choice of making move for his/her opponent

In [31]:
'''
  Three choices are considered for a move (ie) Random, simple and qlearner
  Random move will choose random move from the actions that are available for a particular board state.
  Simple move will choose the action for the state with minimum count.
  Qlearner will choose best action for a state using Qvalues in the QTable
  Random and Simple move are used for explore new states and actions.
  QLearner is for exploitation.
'''
def MakeMove(currentState,choice):
    # possible actions
    possibleActions=list(QTable[currentState].keys())
    #choiceOfMove=['random','simple','qlearner']
    if choice=='random':
        #returning random action from the possible actions.
        return possibleActions[random.randint(0,len(possibleActions)-1)]
        
    if choice=='simple':
        minCount=1000000
        action=0
        for action in possibleActions:
            #VISIT_TABLE stores how many times the states have been visited for each action.
            if VISIT_TABLE[transition(currentState,action)]<minCount:
                minCount=VISIT_TABLE[transition(currentState,action)]
                bestAction=action     
        return bestAction
    if choice=='qlearner':
        return bestActionForState(currentState)  

    
 

### Checks if the game is over

In [ ]:
'''
End state for a board is '111111111111' -> all lines are drawn.
'''
def isBoardFinished(boardState):
    return boardState.bin==12*'1'

### Transition function
- Takes input as current state and move and gives the next state as output

In [25]:
'''
It outputs the next state.
Ex: currentState is '100000000000' and current move is line number 12, then
it will return '100000000001' as the nextstate
'''
def transition(currentState,currentMove):
    temp=currentState
    temp=temp[:currentMove-1]+'1'+temp[currentMove:]
    return temp


### Counts the number of boxes formed during the move to update points

In [26]:
'''
This function will check whether the box is formed or not. If so, it will return the
number of boxes formed for that move. If not, it will return 0 .
  .    1   .    2     . 
  
  7        8          9

  .    3   .     4    .

  10       11         12

  .    5    .   6      .

  Line numbers 1,3,7,8 will form one box and 2,4,8,9 will form another box
  and 3,5,10,11 will form third box and 4,6,11,12 will form the last box
'''
def numberOfNewBoxFormed(box,newState):
    count=0
    boxBounds=[[1,3,7,8],[2,4,8,9],[3,5,10,11],[4,6,11,12]]
    for i in range(len(boxBounds)):
      if box[i]==0:
        flag=True
        for lineNumber in boxBounds[i]:
          if newState[lineNumber-1]!='1':
            flag=False
            break
        if flag==True:
          count+=1
          box[i]=1
    return count
        
                

### Main function where the game is played

Algo
- init board state
- init players as p1,p2 or p1,human
- init current player = p1
- init box [0 0 0 0]
- while not final state: 
    - if cp != 'human' : 
        currentplayer.make move // simple, q learner , random .. 3 boxes..
    - else:
        accept input
    - update currentplayer.memory
    - Check if new box formed, update player score
    - else 
    -   toggle the currentplayer
    - update board state with current move
- update QTABLE with rewards and penalties


In [27]:
'''


'''

def playGame(choiceOfMove,choiceOfPlayer,withHuman=False):
    #Game init
    player=["p1","p2"]
    if withHuman:
        player[1]='human'
    if choiceOfPlayer==2:
      currentPlayer='human'
    else:
      currentPlayer='p1'
    box=[0,0,0,0]
    currentState=BitArray(uint=0, length=12)
    memory={player[0]:[],player[1]:[]}
    score={player[0]:0,player[1]:0}
    while isBoardFinished(currentState)==False:
        if(currentPlayer!="human"):
            currentMove=MakeMove(currentState.bin,choiceOfMove)
        else:
          possibleActions=list(QTable[currentState.bin].keys())
          currentMove=int(input("Enter the line number:"))
          while currentMove not in possibleActions:
            currentMove=int(input("INVALID!Enter the line number:"))

        newState=transition(currentState.bin,currentMove) 
        memory[currentPlayer].append([currentState.bin,currentMove,newState,0])
        numberOfBoxes=numberOfNewBoxFormed(box,newState)
        if numberOfBoxes>0:#Short term rewards
            score[currentPlayer]+=numberOfBoxes*2
            memory[currentPlayer][-1][-1]=100
            memory[player[1-player.index(currentPlayer)]][-1][-1]=-100

        else:
            currentPlayer=player[1-player.index(currentPlayer)]

        currentState=BitArray(uint=int(newState,2),length=12)
        if player[1]=='human':            
            printTable(currentState.bin)
            print("Score of ",player[0]," is ",score[player[0]])
            print("Score of ",player[1]," is ",score[player[1]])
            if isBoardFinished(currentState)==False:
              print("Next turn for ",currentPlayer)

    #find winner 
    result=""
    if score[player[0]]==score[player[1]]:
        result='TIE'
        print(result)
    else:
        result=max(score,key=score.get)
        print("Winner is ",result)
    #print(memory[player[0]])
    if result=='TIE':
        memory[player[0]][-1][-1]=50
        memory[player[1]][-1][-1]=50
    else: 
        memory[result][-1][-1]=200
        memory[player[1-player.index(result)]][-1][-1]=-200
    # print(memory[player[0]])
    # print(memory[player[1]])    
    #put reward and penalties in last row of the memory table
    updateQTable(memory[player[0]])
    updateQTable(memory[player[1]])





### Learn and test performance

In [ ]:
# initialiseQTable()
# learner(gameCount=10000)
print("Enter 1 -> Player 1 : Computer and Player 2: Human")
print("Enter 2 -> Player 1 : Human and Player 2: Computer")
choiceOfPlayer=int(i12nput())
playGame(choiceOfMove='qlearner',choiceOfPlayer=choiceOfPlayer,withHuman=True)

### Save the QTABLE and print visit table


In [ ]:
playGame(choiceOfMove='qlearner',withHuman=True)

with open('Qtable.json','w') as jsonfile:
  json.dump(QTable,jsonfile)

print(VISIT_TABLE)

## Check the performance of our machine by letting it play N games with random agent and find its winrate



In [ ]:
def playGame_AgentVsRandomAgent():

    player=["p1","p2"]    
    currentPlayer="p1"
    box=[0,0,0,0]
    currentState=BitArray(uint=0, length=12)
    memory={player[0]:[],player[1]:[]}
    score={player[0]:0,player[1]:0}
    while isBoardFinished(currentState)==False:
        if(currentPlayer=="p1"):
          currentMove=MakeMove(currentState.bin,'qlearner')
        else:
          currentMove=MakeMove(currentState.bin,'random')
          

        newState=transition(currentState.bin,currentMove) 
        memory[currentPlayer].append([currentState.bin,currentMove,newState,0])
        numberOfBoxes=numberOfNewBoxFormed(box,newState)
        if numberOfBoxes>0:#Short term rewards
            score[currentPlayer]+=numberOfBoxes*2
            memory[currentPlayer][-1][-1]=100
            memory[player[1-player.index(currentPlayer)]][-1][-1]=-100

        else:
            currentPlayer=player[1-player.index(currentPlayer)]

        currentState=BitArray(uint=int(newState,2),length=12)
        
    #find winner 
    result=""
    if score[player[0]]==score[player[1]]:
        result='TIE'
        print(result)
    else:
        result=max(score,key=score.get)
        print("Winner is ",result)
    #print(memory[player[0]])
    if result=='TIE':
        memory[player[0]][-1][-1]=50
        memory[player[1]][-1][-1]=50
    else: 
        memory[result][-1][-1]=200
        memory[player[1-player.index(result)]][-1][-1]=-200
    # print(memory[player[0]])
    # print(memory[player[1]])    
    #put reward and penalties in last row of the memory table
    updateQTable(memory[player[0]])
    updateQTable(memory[player[1]])
    return result

In [ ]:
winCount=0
loseCount=0
N=100
for i in range(1,N+1):
  winner = playGame_AgentVsRandomAgent()
  print('Game ',i," : Winner is ",winner)
  if winner =='p1':
    winCount+=1
  if winner =='p2':
    loseCount+=1


print("Of the ",N," games, our Q Learner  has winrate =",winCount/N," and loserate =",loseCount/N,". It has tie rate of  ", (N-winCount-loseCount)/N ) 

